# Amazon KDD Cup 2023 - Task 1 - Next Product Recommendation 

![](https://images.aicrowd.com/raw_images/challenges/banner_file/1116/6c8fecd6d7c225b4ed11.jpg)

This notebook will contains instructions and example submission with random predictions.



## Installations 🤖

1. `aicrowd-cli` for downloading challenge data and making submissions
2. `pyarrow` for saving to parquet for submissions

In [1]:
# %pip install aicrowd-cli pyarrow

## Login to AIcrowd and download the data 📚

In [2]:
# !aicrowd login

In [3]:
# !aicrowd dataset download --challenge task-1-next-product-recommendation

## Setup data and task information

In [4]:
import os
import numpy as np
import pandas as pd
from functools import lru_cache

In [5]:
# from aicrowd import dataset
# dataset.download_dataset("task-1-next-product-recommendation", "./", 3, [])
# dataset.list_dataset("task-1-next-product-recommendation")

In [6]:
train_data_dir = '.'
test_data_dir = '.'
task = 'task1'
PREDS_PER_SESSION = 100

In [7]:
# Cache loading of data for multiple calls

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(os.path.join(train_data_dir, 'products_train.csv'))

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(os.path.join(train_data_dir, 'sessions_train.csv'))

@lru_cache(maxsize=3)
def read_test_data(task):
    return pd.read_csv(os.path.join(test_data_dir, f'sessions_test_{task}.csv'))

## Data Description

The Multilingual Shopping Session Dataset is a collection of **anonymized customer sessions** containing products from six different locales, namely English, German, Japanese, French, Italian, and Spanish. It consists of two main components: **user sessions** and **product attributes**. User sessions are a list of products that a user has engaged with in chronological order, while product attributes include various details like product title, price in local currency, brand, color, and description.

---

### Each product as its associated information:


**locale**: the locale code of the product (e.g., DE)

**id**: a unique for the product. Also known as Amazon Standard Item Number (ASIN) (e.g., B07WSY3MG8)

**title**: title of the item (e.g., “Japanese Aesthetic Sakura Flowers Vaporwave Soft Grunge Gift T-Shirt”)

**price**: price of the item in local currency (e.g., 24.99)

**brand**: item brand name (e.g., “Japanese Aesthetic Flowers & Vaporwave Clothing”)

**color**: color of the item (e.g., “Black”)

**size**: size of the item (e.g., “xxl”)

**model**: model of the item (e.g., “iphone 13”)

**material**: material of the item (e.g., “cotton”)

**author**: author of the item (e.g., “J. K. Rowling”)

**desc**: description about a item’s key features and benefits called out via bullet points (e.g., “Solid colors: 100% Cotton; Heather Grey: 90% Cotton, 10% Polyester; All Other Heathers …”)


## EDA 💽

In [8]:
def read_locale_data(locale, task):
    products = read_product_data().query(f'locale == "{locale}"')
    sess_train = read_train_data().query(f'locale == "{locale}"')
    sess_test = read_test_data(task).query(f'locale == "{locale}"')
    return products, sess_train, sess_test

def show_locale_info(locale, task):
    products, sess_train, sess_test = read_locale_data(locale, task)

    train_l = sess_train['prev_items'].apply(lambda sess: len(sess))
    test_l = sess_test['prev_items'].apply(lambda sess: len(sess))

    print(f"Locale: {locale} \n"
          f"Number of products: {products['id'].nunique()} \n"
          f"Number of train sessions: {len(sess_train)} \n"
          f"Train session lengths - "
          f"Mean: {train_l.mean():.2f} | Median {train_l.median():.2f} | "
          f"Min: {train_l.min():.2f} | Max {train_l.max():.2f} \n"
          f"Number of test sessions: {len(sess_test)}"
        )
    if len(sess_test) > 0:
        print(
             f"Test session lengths - "
            f"Mean: {test_l.mean():.2f} | Median {test_l.median():.2f} | "
            f"Min: {test_l.min():.2f} | Max {test_l.max():.2f} \n"
        )
    print("======================================================================== \n")

In [9]:
products = read_product_data()
locale_names = products['locale'].unique()
for locale in locale_names:
    show_locale_info(locale, task)

Locale: DE 
Number of products: 518327 
Number of train sessions: 1111416 
Train session lengths - Mean: 57.89 | Median 40.00 | Min: 27.00 | Max 2060.00 
Number of test sessions: 104568
Test session lengths - Mean: 57.23 | Median 40.00 | Min: 27.00 | Max 700.00 


Locale: JP 
Number of products: 395009 
Number of train sessions: 979119 
Train session lengths - Mean: 59.61 | Median 40.00 | Min: 27.00 | Max 6257.00 
Number of test sessions: 96467
Test session lengths - Mean: 59.90 | Median 40.00 | Min: 27.00 | Max 1479.00 


Locale: UK 
Number of products: 500180 
Number of train sessions: 1182181 
Train session lengths - Mean: 54.85 | Median 40.00 | Min: 27.00 | Max 2654.00 
Number of test sessions: 115936
Test session lengths - Mean: 53.51 | Median 40.00 | Min: 27.00 | Max 872.00 


Locale: ES 
Number of products: 42503 
Number of train sessions: 89047 
Train session lengths - Mean: 48.82 | Median 40.00 | Min: 27.00 | Max 792.00 
Number of test sessions: 0

Locale: FR 
Number of produc

In [10]:
products.sample(5)

,id,locale,title,price,brand,color,size,model,material,author,desc
1378635,B09NYJKGCT,UK,Design By Violet KOALA 'THANK YOU' SINGLE GREE...,0.49,Design By Violet,NaN,One Size,DBV-83-SC142,NaN,NaN,Blank inside for you to write your own thought...
679281,B09YCLJKMG,JP,Xperia 10 IV ケース クリア TPU 耐衝撃 Xperia 10 IV SO-5...,794.00,yaoshengj,クリア,Xperia 10 IV,NaN,シリコーン,NaN,【ワイヤレス充電対応】ケースが薄型なので、ケースをつけた状態でいつでも保護しつつ、ワイヤレス...
1385711,B082T62BCF,UK,Amazon Basics Nylon Braided Lightning to USB A...,17.84,Amazon Basics,Dark Grey,1.8 m,L6LMF894-CS-R,NaN,NaN,Added Protection: An additional layer of prote...
962286,B08NFH7ZD7,UK,Iris & Lilly Women's Cotton Cheeky Hipster Kni...,9.40,IRIS & LILLY,Grey Heather/Dark Navy,14,BELD006M5,Cotton,NaN,An Amazon brand
1181393,B08XZ3FRN8,UK,"Joomer 2 Pack Halloween Twinkle Fairy Lights, ...",16.99,Joomer,Orange,50LED Orange,NaN,NaN,NaN,"8 Lighting Modes: Features combination, wave, ..."


In [11]:
train_sessions = read_train_data()
train_sessions.sample(5)

,prev_items,next_item,locale
3292526,['B09BB8JM7J' 'B09PZT417Y' 'B09Y9639DB'],B09MTLVMJC,ES
266743,['B08VYZS786' 'B08VZCBWN8'],B08VYX257R,DE
2126492,['B08HRC48Y5' 'B08HRCJDMB' 'B09CLNWWF5'],B09CLL3WR4,UK
580942,['B0BHZXWDMK' 'B0BHZX4WC6' 'B01DQXP9A8' 'B0B81...,B08CNK4LLJ,DE
3328328,['B0BFDFR5JR' 'B01M3122UD' 'B07J5WHCNK' 'B08HQ...,B07JK4JT7X,ES


In [12]:
test_sessions = read_test_data(task)
test_sessions.sample(5)

,prev_items,locale
126475,['B0873GSWFY' 'B07L44RHC2'],JP
267877,['B09P9ZPFP3' 'B08CN43GRM'],UK
75696,['B004FPWAVI' 'B004FPWAVI' 'B09N7FVNFF' 'B09N7...,DE
230870,['B092S5DJ4F' 'B09TDTK9G9' 'B09TDLQSKK'],UK
299886,['B0BJ9F9LGZ' 'B09SWS377M' 'B09SWS377M' 'B09TB...,UK


## Generate Submission 🏋️‍♀️



Submission format:
1. The submission should be a **parquet** file with the sessions from all the locales. 
2. Predicted products ids per locale should only be a valid product id of that locale. 
3. Predictions should be added in new column named **"next_item_prediction"**.
4. Predictions should be a list of string id values

In [13]:
def random_predicitons(locale, sess_test_locale):
    random_state = np.random.RandomState(42)
    products = read_product_data().query(f'locale == "{locale}"')
    predictions = []
    for _ in range(len(sess_test_locale)):
        predictions.append(
            list(products['id'].sample(PREDS_PER_SESSION, replace=True, random_state=random_state))
        ) 
    sess_test_locale['next_item_prediction'] = predictions
    sess_test_locale.drop('prev_items', inplace=True, axis=1)
    return sess_test_locale

In [14]:
test_sessions = read_test_data(task)
predictions = []
test_locale_names = test_sessions['locale'].unique()
for locale in test_locale_names:
    sess_test_locale = test_sessions.query(f'locale == "{locale}"').copy()
    predictions.append(
        random_predicitons(locale, sess_test_locale)
    )
predictions = pd.concat(predictions).reset_index(drop=True)
predictions.sample(5)

,locale,next_item_prediction
33716,DE,"[B07MPSKJXW, B00FDRMCW4, B08D3BNX9Q, B0BKV1ZV2..."
41473,DE,"[B09Z316N7D, B07P8HWKGV, B07BSD8M5N, B0BHDRXTT..."
173564,JP,"[B0B99XMM1D, B09956MYFX, B07BZ3QRHS, B07F68LXG..."
64238,DE,"[B093Q4XZGY, B07KYXQYJ8, B008KJNAPA, B08LQGKNM..."
39193,DE,"[3060330891, B0B2NMNJJV, B081WZRXVX, B09KV1P1W..."


# Validate predictions ✅

In [15]:
def check_predictions(predictions, check_products=False):
    """
    These tests need to pass as they will also be applied on the evaluator
    """
    test_locale_names = test_sessions['locale'].unique()
    for locale in test_locale_names:
        sess_test = test_sessions.query(f'locale == "{locale}"')
        preds_locale =  predictions[predictions['locale'] == sess_test['locale'].iloc[0]]
        assert sorted(preds_locale.index.values) == sorted(sess_test.index.values), f"Session ids of {locale} doesn't match"

        if check_products:
            # This check is not done on the evaluator
            # but you can run it to verify there is no mixing of products between locales
            # Since the ground truth next item will always belong to the same locale
            # Warning - This can be slow to run
            products = read_product_data().query(f'locale == "{locale}"')
            predicted_products = np.unique( np.array(list(preds_locale["next_item_prediction"].values)) )
            assert np.all( np.isin(predicted_products, products['id']) ), f"Invalid products in {locale} predictions"

In [24]:
check_predictions(predictions)
predictions.sample(5)

,locale,next_item_prediction
153422,JP,"[B073SVWVCW, B001BDZGT0, B078DL2DB9, B07HCSBX4..."
252343,UK,"[B01H7N8NMO, B086DSV2FK, B087QJHZMV, B07N854CW..."
125244,JP,"[B000QUW426, B0B931JL1F, B08BC3JLZ5, B07N12QZX..."
136901,JP,"[B0B6B68G3B, B071FGYLP9, B092V3WY2R, B078HQ88H..."
234770,UK,"[B095Y9NWVM, B07PFX459R, B0B38RH4Q5, B0B1Q7QZ9..."


In [17]:
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(f'submission_{task}.parquet', engine='pyarrow')

## Submit to AIcrowd 🚀

In [18]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
# !aicrowd submission create -c task-1-next-product-recommendation -f "submission_task1.parquet"